In [2]:
import pandas as pd
import os
import glob
from PIL import Image, ImageDraw, ImageFont
import re

In [3]:
%cd ./experiments

c:\Users\Ahouz\OneDrive\Bureau\PGM_papers\PGM_project\experiments


### Read the data containing different experiments 

In [ ]:
# folders are writen as 10_samples_100_steps_sigma_10.0000_1.0000_eps_0.01000_pretrained we scan for the pretrained folder and store parameters with corresponding folder name
folders = os.listdir()
folders = [folder for folder in folders if folder.endswith('pretrained')]
 
# create empty dataframe
df = pd.DataFrame(columns=['samples', 'steps', 'sigma_max', 'sigma_min', 'eps', 'folder'])
    
# loop over folders and store parameters
for folder in folders:
    split = folder.split('_')
    samples, steps, sigma_max,sigma_min, eps = split[0], split[2], split[5], split[6], split[8]
    df = df.append({'samples': samples, 'steps': steps, 'sigma_max': sigma_max, 'sigma_min': sigma_min, 'eps': eps, 'folder': folder, 'steps': steps}, ignore_index=True)
# sort dataframe
    
df = df.sort_values(by=['samples', 'steps', 'sigma_max', 'sigma_min', 'eps'])
    
     

### Effect of epsilon on the genrated images

In [21]:
# first we need to create the folders
eps_folders = df[df['sigma_max'] == '1.0000']
eps_folders = eps_folders[eps_folders['sigma_min'] == '0.0010']
eps_folders_eps = eps_folders['eps'].unique()
if not os.path.exists('../effects_eps'):
    os.mkdir('../effects_eps')
for eps in eps_folders_eps:
    if not os.path.exists('../effects_eps/eps_' + eps):
        os.mkdir('../effects_eps/eps_' + eps)


In [22]:
# now we need to create the gifs
for eps in eps_folders_eps:
    # get folder for this eps
    eps_folder = eps_folders[eps_folders['eps'] == eps]['folder']
    # create gif for each folder
    for folder in eps_folder:
        # get all images
        images = glob.glob(folder + '/*.png')
        # start from highest sigma to lower then sort by lowest step
        images = sorted(images, key=lambda x: (-float(re.search(r'sigma_(\d+\.\d+)', x.split("\\")[1]).group(1)), int(re.search(r'step_([\d]+)', x.split("\\")[1]).group(1))))  
        # create gif
        gif_path = '../effects_eps/eps_' + eps + '/' + folder + '.gif'
        gif_images = [Image.open(image) for image in images]
        # delete all files image_*.png
        for image in images:
            # if image string is image_*.png
            if re.search(r'image_([\d]+).png', image.split("\\")[1]):
                os.remove(image)   
        # extract step and sigma from image name
        step_str_table = [int(re.search(r'step_([\d]+)', image.split("\\")[1]).group(1)) for image in images]
        sigma_str_table = [re.search(r'sigma_(\d+\.\d+)', image.split("\\")[1]).group(1) for image in images]   
        # create captions
        captions = ['step: ' + str(step) + ' sigma: ' + str(sigma) for step, sigma in zip(step_str_table, sigma_str_table)]
        # add captions to images
        images_with_captions = []
        for i, image in enumerate(images):
            img = Image.open(image)
            font = ImageFont.load_default()
            # fill the extra space with white and paste the old img on the original position
            big_img = Image.new('RGB', (img.width, img.height + 20), color = (255, 255, 255))
            big_img.paste(img, (0, 0))  
            draw = ImageDraw.Draw(big_img)
            draw.text((0, img.height), captions[i], (0,0,0), font=font)
            images_with_captions.append(big_img)      
        # save gif
        images_with_captions[0].save(gif_path, save_all=True, append_images=images_with_captions[:], duration=1000, loop=0)

### Effect of sigma min

In [23]:
# we get the folders with epsilon=0.00005 and sigma_max=1.0000 
# the sigma_min will vary
sigma_max_folders = df[df['eps'] == '0.00005']
sigma_max_folders = sigma_max_folders[sigma_max_folders['sigma_max'] == '1.0000']
sigma_max_folders_sigma_min = sigma_max_folders['sigma_min'].unique()
if not os.path.exists('../effects_sigma_min'):
    os.mkdir('../effects_sigma_min')
for sigma_min in sigma_max_folders_sigma_min:
    if not os.path.exists('../effects_sigma_min/sigma_min_' + sigma_min):
        os.mkdir('../effects_sigma_min/sigma_min_' + sigma_min)
            

In [24]:
# now we need to create the gifs
for sigma_min in sigma_max_folders_sigma_min:
    # get folder for this eps
    sigma_min_folder = sigma_max_folders[sigma_max_folders['sigma_min'] == sigma_min]['folder']
    print(sigma_min_folder)
    # create gif for each folder
    for folder in sigma_min_folder:
        # get all images
        images = glob.glob(folder + '/*.png')
        # start from highest sigma to lower then sort by lowest step
        images = sorted(images, key=lambda x: (-float(re.search(r'sigma_(\d+\.\d+)', x.split("\\")[1]).group(1)), int(re.search(r'step_([\d]+)', x.split("\\")[1]).group(1))))
        # create gif
        gif_path = '../effects_sigma_min/sigma_min_' + sigma_min + '/' + folder + '.gif'
        gif_images = [Image.open(image) for image in images]
        # delete all files image_*.png
        for image in images:
            # if image string is image_*.png
            if re.search(r'image_([\d]+).png', image.split("\\")[1]):
                os.remove(image)
        # extract step and sigma from image name
        step_str_table = [int(re.search(r'step_([\d]+)', image.split("\\")[1]).group(1)) for image in images]
        sigma_str_table = [re.search(r'sigma_(\d+\.\d+)', image.split("\\")[1]).group(1) for image in images]
        # create captions
        captions = ['step: ' + str(step) + ' sigma: ' + str(sigma) for step, sigma in zip(step_str_table, sigma_str_table)]
        # add captions to images
        images_with_captions = []
        for i, image in enumerate(images):
            img = Image.open(image)
            font = ImageFont.load_default()
            # fill the extra space with white and paste the old img on the original position
            big_img = Image.new('RGB', (img.width, img.height + 20), color = (255, 255, 255))
            big_img.paste(img, (0, 0))  
            draw = ImageDraw.Draw(big_img)
            draw.text((0, img.height), captions[i], (0,0,0), font=font)
            images_with_captions.append(big_img)
             
        # save gif
        images_with_captions[0].save(gif_path, save_all=True, append_images=images_with_captions[:], duration=1000, loop=0)

6     10_samples_100_steps_sigma_1.0000_0.0010_eps_0...
79    10_samples_150_steps_sigma_1.0000_0.0010_eps_0...
80    10_samples_200_steps_sigma_1.0000_0.0010_eps_0...
81    10_samples_50_steps_sigma_1.0000_0.0010_eps_0....
Name: folder, dtype: object
13    10_samples_100_steps_sigma_1.0000_0.0100_eps_0...
Name: folder, dtype: object
21    10_samples_100_steps_sigma_1.0000_0.1000_eps_0...
Name: folder, dtype: object
29    10_samples_100_steps_sigma_1.0000_1.0000_eps_0...
Name: folder, dtype: object


### Effect of sigma max

In [25]:
# we get the folders with epsilon=0.00005 and sigma_min=0.0010
# the sigma_max will vary
sigma_min_folders = df[df['eps'] == '0.00005']
sigma_min_folders = sigma_min_folders[sigma_min_folders['sigma_min'] == '0.0010']
sigma_min_folders_sigma_max = sigma_min_folders['sigma_max'].unique()
if not os.path.exists('../effects_sigma_max'):
    os.mkdir('../effects_sigma_max')
for sigma_max in sigma_min_folders_sigma_max:
    if not os.path.exists('../effects_sigma_max/sigma_max_' + sigma_max):
        os.mkdir('../effects_sigma_max/sigma_max_' + sigma_max)
    

In [26]:
# now we need to create the gifs 
    
for sigma_max in sigma_min_folders_sigma_max:
    # get folder for this eps
    sigma_max_folder = sigma_min_folders[sigma_min_folders['sigma_max'] == sigma_max]['folder']
    print(sigma_max_folder)
    # create gif for each folder
    for folder in sigma_max_folder:
        # get all images
        images = glob.glob(folder + '/*.png')
        # start from highest sigma to lower then sort by lowest step
        images = sorted(images, key=lambda x: (-float(re.search(r'sigma_(\d+\.\d+)', x.split("\\")[1]).group(1)), int(re.search(r'step_([\d]+)', x.split("\\")[1]).group(1))))  
        # create gif
        gif_path = '../effects_sigma_max/sigma_max_' + sigma_max + '/' + folder + '.gif'
        gif_images = [Image.open(image) for image in images]
        # delete all files image_*.png
        for image in images:
            # if image string is image_*.png
            if re.search(r'image_([\d]+).png', image.split("\\")[1]):
                os.remove(image)
        # extract step and sigma from image name
        step_str_table = [int(re.search(r'step_([\d]+)', image.split("\\")[1]).group(1)) for image in images]
        sigma_str_table = [re.search(r'sigma_(\d+\.\d+)', image.split("\\")[1]).group(1) for image in images]
        # create captions
        captions = ['step: ' + str(step) + ' sigma: ' + str(sigma) for step, sigma in zip(step_str_table, sigma_str_table)]
        # add captions to images
        images_with_captions = []
        for i, image in enumerate(images):
            img = Image.open(image)
            font = ImageFont.load_default()
            # fill the extra space with white and paste the old img on the original position
            big_img = Image.new('RGB', (img.width, img.height + 20), color = (255, 255, 255))
            big_img.paste(img, (0, 0))  
            draw = ImageDraw.Draw(big_img)
            draw.text((0, img.height), captions[i], (0,0,0), font=font)
            images_with_captions.append(big_img)
             
        # save gif
        images_with_captions[0].save(gif_path, save_all=True, append_images=images_with_captions[:], duration=1000, loop=0)

0    10_samples_100_steps_sigma_0.0100_0.0010_eps_0...
Name: folder, dtype: object
1    10_samples_100_steps_sigma_0.1000_0.0010_eps_0...
Name: folder, dtype: object
6     10_samples_100_steps_sigma_1.0000_0.0010_eps_0...
79    10_samples_150_steps_sigma_1.0000_0.0010_eps_0...
80    10_samples_200_steps_sigma_1.0000_0.0010_eps_0...
81    10_samples_50_steps_sigma_1.0000_0.0010_eps_0....
Name: folder, dtype: object
45    10_samples_100_steps_sigma_10.0000_0.0010_eps_...
Name: folder, dtype: object
75    10_samples_100_steps_sigma_2.0000_0.0010_eps_0...
Name: folder, dtype: object
76    10_samples_100_steps_sigma_4.0000_0.0010_eps_0...
Name: folder, dtype: object
77    10_samples_100_steps_sigma_6.0000_0.0010_eps_0...
Name: folder, dtype: object
78    10_samples_100_steps_sigma_8.0000_0.0010_eps_0...
Name: folder, dtype: object


### Effect of step :

In [27]:
#get the folders with sigma_max=1.0000 and sigma_min=0.0010 and epsilon=0.00005
# the number of steps will vary
steps_folders = df[df['eps'] == '0.00005']
steps_folders = steps_folders[steps_folders['sigma_min'] == '0.0010']
steps_folders = steps_folders[steps_folders['sigma_max'] == '1.0000']
steps_folders_steps = steps_folders['steps'].unique()
if not os.path.exists('../effects_steps'):
    os.mkdir('../effects_steps')
for steps in steps_folders_steps:
    if not os.path.exists('../effects_steps/steps_' + steps):
        os.mkdir('../effects_steps/steps_' + steps)
         

In [28]:
steps_folders_steps

array(['100', '150', '200', '50'], dtype=object)

In [29]:
# now we need to create the gifs
for steps in steps_folders_steps:
    # get folder for this eps
    steps_folder = steps_folders[steps_folders['steps'] == steps]['folder']
    # create gif for each folder
    for folder in steps_folder:
        # get all images
        images = glob.glob(folder + '/*.png')
        # start from highest sigma to lower then sort by lowest step
        images = sorted(images, key=lambda x: (-float(re.search(r'sigma_(\d+\.\d+)', x.split("\\")[1]).group(1)), int(re.search(r'step_([\d]+)', x.split("\\")[1]).group(1))))  
        # create gif
        gif_path = '../effects_steps/steps_' + steps + '/' + folder + '.gif'
        gif_images = [Image.open(image) for image in images]
        # delete all files image_*.png
        for image in images:
            # if image string is image_*.png
            if re.search(r'image_([\d]+).png', image.split("\\")[1]):
                os.remove(image)
        # extract step and sigma from image name
        step_str_table = [int(re.search(r'step_([\d]+)', image.split("\\")[1]).group(1)) for image in images]
        sigma_str_table = [re.search(r'sigma_(\d+\.\d+)', image.split("\\")[1]).group(1) for image in images]
        # create captions
        captions = ['step: ' + str(step) + ' sigma: ' + str(sigma) for step, sigma in zip(step_str_table, sigma_str_table)]
        # add captions to images
        images_with_captions = []
        for i, image in enumerate(images):
            img = Image.open(image)
            font = ImageFont.load_default()
            # fill the extra space with white and paste the old img on the original position
            big_img = Image.new('RGB', (img.width, img.height + 20), color = (255, 255, 255))
            big_img.paste(img, (0, 0))  
            draw = ImageDraw.Draw(big_img)
            draw.text((0, img.height), captions[i], (0,0,0), font=font)
            images_with_captions.append(big_img)
             
        # save gif
        images_with_captions[0].save(gif_path, save_all=True, append_images=images_with_captions[:], duration=1000, loop=0)